In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import utils
import json
import os
import git
import warnings
warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

## Get the necessary data from Hopsworks

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
# secrets = utils.secrets_api(project.name)

CITY = "dublin"
STATION = "HEUSTON BRIDGE (NORTH)"

# latitude =
# longitude =

today = datetime.datetime.now()

2025-01-20 09:46:36,815 INFO: Initializing external client
2025-01-20 09:46:36,816 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-20 09:46:38,970 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494


In [4]:
# Retrieve feature groups
bike_fg = fs.get_feature_group(
    name='bike_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)


## Clone and pull the repository with the bike data

In [5]:
# Configuration
REPO_URL = "https://github.com/MaxHalford/bike-sharing-history"
CLONE_DIR = "./bike_data/bike-sharing-history"
TARGET_CITY = "dublin"
FILE_NAME = "jcdecaux.geojson"

In [6]:
if not os.path.exists(CLONE_DIR):
    print("Cloning repository...")
    git.Repo.clone_from(REPO_URL, CLONE_DIR)
repo = git.Repo(CLONE_DIR)

# go to main and pull the latest changes
repo.git.checkout("main", force=True)
repo.remotes.origin.pull()

## Get the latest datetime present in the bike data

In [7]:
# last_bike_datetime = "2025-01-06 15:06:11 UTC"

bike_df = bike_fg.read()

last_bike_datetime = bike_df["datetime"].max()
last_bike_datetime = last_bike_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")

last_bike_datetime

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.29s) 


'2025-01-10 23:49:13 UTC'

## Loop through the commits and convert the bike data into a dataframe

In [8]:
results = {}

# Populate the results dict with the stations
data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
if os.path.exists(data_file):
    with open(data_file, "r") as f:
        data = f.read()
        data = json.loads(data)
        for feature in data["features"]:
            name = feature["properties"]["name"]
            results[name] = []

In [9]:
start_time = datetime.datetime.now()
last_day_and_hour = None

for commit in repo.iter_commits():
    # Stop when we reach the earliest bike date
    if commit.committed_datetime <= datetime.datetime.strptime(last_bike_datetime, "%Y-%m-%d %H:%M:%S %Z").replace(tzinfo=datetime.timezone.utc):
        print("breaking at: ", commit.committed_datetime)
        break

    # Skip commits from today
    if commit.committed_datetime > today.replace(tzinfo=datetime.timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0):
        first_commited_datetime = repo.commit().committed_datetime
        continue

    day_and_hour = commit.committed_datetime.replace(minute=0, second=0, microsecond=0)
    # print("day_and_hour: ", day_and_hour, " - last_day_and_hour: ", last_day_and_hour)
    if day_and_hour == last_day_and_hour:
        continue
    last_day_and_hour = day_and_hour

    # Get the data for the commit
    print("Processing commit: ", commit.hexsha, " - ", commit.committed_datetime)
    try:
        repo.git.checkout(commit.hexsha, force=True)
    except Exception as e:
        print("Error checking out commit: ", e)
        break

    data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
    if os.path.exists(data_file):
        with open(data_file, "r") as f:
            data = f.read()
            data = json.loads(data)
            for feature in data["features"]:
                try:
                    results[feature["properties"]["name"]].append([feature["properties"]["available_bikes"], commit.committed_datetime])
                except KeyError:
                    continue

print(results)


Processing commit:  3690701071b9fdcadbae577b247ce8b22dd44e9f  -  2025-01-19 23:49:13+00:00
Processing commit:  66c94f5391c23374ac08b2cd61416fe18cae7f59  -  2025-01-19 22:49:03+00:00
Processing commit:  427320ca389472ea0233e9fe50e6af1533c8caa8  -  2025-01-19 21:49:00+00:00
Processing commit:  dc8be25531e80708d2ff5ec5d914540287c74a5d  -  2025-01-19 20:49:10+00:00
Processing commit:  6a77b2901539fae5af0cf2b18a6be81e54cbeef0  -  2025-01-19 19:49:09+00:00
Processing commit:  0428e3b89c8ce064e14e7bd0441faca2f63cd3a6  -  2025-01-19 18:48:59+00:00
Processing commit:  783b105f922630b28d2a4212509f00c8b0457582  -  2025-01-19 17:48:59+00:00
Processing commit:  885074b8f9b4f281dfbe97e83459cac96eef007c  -  2025-01-19 16:49:07+00:00
Processing commit:  4c89b6a45a1b397e32d162d70ee80a08c1babdc9  -  2025-01-19 15:48:59+00:00
Processing commit:  f33408351dfe2885375b78e6e4f118fe2a9b0f26  -  2025-01-19 14:49:06+00:00
Processing commit:  ef0307bfd210b49604429c27edb7ecb9cbaddbec  -  2025-01-19 13:49:02+00:00

In [10]:
# turn results into a dataframe
df_bike_today = pd.DataFrame()

for station, values in results.items():
    if len(values) > 0:
        df = pd.DataFrame(values, columns=["num_bikes_available", "datetime"])
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
        df["station"] = station.replace(" ", "_")
        df_bike_today = pd.concat([df_bike_today, df])

# if empty, do nothing
if not df_bike_today.empty:
    df_bike_today.dropna(inplace=True)
    df_bike_today["num_bikes_available"] = df_bike_today["num_bikes_available"].astype("float32")
    df_bike_today = df_bike_today[df_bike_today['station'].isin([STATION.replace(" ", "_")])]
df_bike_today


## Fetch the weather data for the same time period

In [11]:
forecast_df = utils.get_hourly_weather_forecast(CITY)
forecast_df = forecast_df.rename(columns={'date_x': 'datetime'})
forecast_df = forecast_df.drop(columns=['date_y', 'date_only'])
forecast_df.dropna(inplace=True)

print(forecast_df.empty)

forecast_df

features: {'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
params: {'latitude': 53.35, 'longitude': -6.26, 'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
Coordinates 53.5°N -6.25°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
False


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2025-01-20 00:00:00+00:00,7.40,5.370594,0.0,0.0,7.594207,29881.912109,1.8,dublin
1,2025-01-20 01:00:00+00:00,7.55,5.414439,0.1,0.0,8.557102,29881.912109,1.8,dublin
2,2025-01-20 02:00:00+00:00,7.75,5.377898,0.1,0.0,10.495713,29881.912109,1.8,dublin
3,2025-01-20 03:00:00+00:00,7.90,5.379412,0.1,0.0,11.753877,29881.912109,1.8,dublin
4,2025-01-20 04:00:00+00:00,7.90,5.365732,0.2,0.0,12.101570,29881.912109,1.8,dublin
...,...,...,...,...,...,...,...,...,...
235,2025-01-29 19:00:00+00:00,3.10,-0.518530,0.0,0.0,11.681987,31637.470703,0.0,dublin
236,2025-01-29 20:00:00+00:00,3.15,-0.574576,0.0,0.0,11.918120,31637.470703,0.0,dublin
237,2025-01-29 21:00:00+00:00,3.30,-0.619211,0.0,0.0,12.864649,31637.470703,0.0,dublin
238,2025-01-29 22:00:00+00:00,3.55,-0.637305,0.0,0.0,14.408997,31637.470703,0.0,dublin


## Insert the bike and weather data into Hopsworks

In [12]:
if df_bike_today.empty:
    print("No bike data available for today")
else:
    bike_fg.insert(df_bike_today)

Uploading Dataframe: 100.00% |██████████| Rows 216/216 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: bike_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/bike_data_1_offline_fg_materialization/executions


In [ ]:
if forecast_df.empty:
    print("No weather forecast available for today")
else:
    weather_fg.insert(forecast_df, write_options={"wait_for_job": True})


Uploading Dataframe: 100.00% |██████████| Rows 240/240 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions
2025-01-20 09:52:07,637 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-01-20 09:52:10,844 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
